# Übungen zur Klassifikation mit kNN

### Aufgabe 1: Theorie

**a) Was ist die Gefahr, wenn k zu groß bzw. zu klein gesetzt wird?**

Ein großes k reduziert den Einfluss von verrauschten Daten, kann jedoch das Modell so verzerren, dass kleine, aber wichtige Muster übersehen werden. Wenn beispielsweise k so groß gesetzt wird wie die Anzahl der Beobachtungen, dann wäre jede Instanz bei der Abstimmung repräsentiert und das Modell würde immer die Klasse, in der am meisten Instanzen repräsentiert sind, vorhersagen.  
Das entgegengesetzte Extrem, wenn also k=1 gesetzt wird, hat zur Folge, dass verrauschte Daten oder Ausreißer zu starken Einfluss auf die Vorhersage haben.

**b) Warum müssen Daten transfomiert bzw. normalisiert werden?**

 Die Daten müssen ein ein Einheitsintervall transformiert werden, da sonst Merkmale die einen breiten Range haben, viel mehr Einfluss haben, als Merkmale, die ihre Ausprägungen in einem kleinen Intervall haben. Die Gefahr besteht, dass wenn die Merkmale so unterschiedlich sind, dass die Klassifikation nur anhand des Merkmals mit breitem Range stattfindet.

### Aufgabe 2: 

a) Verwenden Sie den Datensatz `Smarket` aus dem package `ISLR`, um ein Modell zu generieren,  mit dem vorhergesagt werden kann, ob der Aktienmarkt steigt (up) oder fällt (down).  Wie viel % werden korrekt vorher gesagt?

**Hinweise**:  
- beachten Sie, dass Sie die Zielvariable als Faktor konvertieren
- Sie sollten die Reihenfolge vorher randomisieren um sicherzugehen, dass der Trainingsdatensatz und Testdatensatz für den Gesamtdatensatz repräsentativ sind.  Dies können Sie wie folgt tun:  
`set.seed(1) # stellt die gleichen Ergebnisse sicher
rnum<- sample(rep(1:1250)) #generiert zufällige Zahlen zwischen 1 und 150
stockm<- stockm[rnum,] #Reihenfolge randomisieren`

- Teilen Sie dann die ersten 1000 Datensätze den Trainingsdaten und die restlichen den Testdaten zu.  

b) Können Sie Ihr Modell evtl. verbessern mit einer anderen Transformation? Wird es besser, wenn sie k = 1 setzten?

## Lösung

Zunächst muss das Package `ISLR` und, falls noch nicht geschehen `class`, installiert werden und die libraries `class` und `ISLR` eingebunden werden. 

In [12]:
library("class")

In [20]:
install.packages('ISLR');
library(ISLR)

package 'ISLR' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\HS\AppData\Local\Temp\Rtmp02bbU8\downloaded_packages


Zuerst kann man sich die ersten Zeilen ansehen und sich mit `str()`einen Überblick über die Daten verschaffen

In [2]:
head(Smarket)

Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
2001,0.213,0.614,-0.623,1.032,0.959,1.3491,1.392,Up


In [3]:
#Datensatz in Dataframe speichern
stockm <-Smarket

In [4]:
str(stockm)

'data.frame':	1250 obs. of  9 variables:
 $ Year     : num  2001 2001 2001 2001 2001 ...
 $ Lag1     : num  0.381 0.959 1.032 -0.623 0.614 ...
 $ Lag2     : num  -0.192 0.381 0.959 1.032 -0.623 ...
 $ Lag3     : num  -2.624 -0.192 0.381 0.959 1.032 ...
 $ Lag4     : num  -1.055 -2.624 -0.192 0.381 0.959 ...
 $ Lag5     : num  5.01 -1.055 -2.624 -0.192 0.381 ...
 $ Volume   : num  1.19 1.3 1.41 1.28 1.21 ...
 $ Today    : num  0.959 1.032 -0.623 0.614 0.213 ...
 $ Direction: Factor w/ 2 levels "Down","Up": 2 2 1 2 2 2 1 2 2 2 ...


Der Datensatz ist nach Jahren sortiert, wir werden die Reihenfolge nun randomisieren.

In [5]:
set.seed(1) # stellt die gleichen Ergebnisse sicher
rnum<- sample(rep(1:1250)) #generiert zufällige Zahlen zwischen 1 und 150
stockm<- stockm[rnum,]

Nun kann übrepüft werden, ob die Randomisation geklappt hat.

In [6]:
head(stockm)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
332,2002,-1.455,3.750,-0.302,-1.934,-1.026,1.15300,-1.679,Down
465,2002,0.615,2.463,-0.048,0.772,-2.073,1.40010,-1.041,Down
715,2003,-0.011,1.143,-0.052,-0.579,-0.457,1.38300,-0.762,Down
1133,2005,0.262,0.088,0.227,0.625,1.168,2.04871,0.116,Up
252,2002,-0.628,-0.947,0.122,-0.480,-0.359,1.28640,0.683,Up
1119,2005,-1.083,0.022,-0.205,-0.071,0.495,2.02992,-0.763,Down


Das sieht gut aus.   
Nun kann man sehen wir uns die Verteilung der Zielvariablen an.

In [45]:
table(stockm$Direction)
round(prop.table(table(stockm$Direction))*100, digits =1)


Down   Up 
 602  648 


Down   Up 
48.2 51.8 

Es gibt in jeder Klasse fast gleich viele Repräsentanten. Die Zielvariable muss noch als Faktor konvertiert werden.

In [8]:
stockm$Direction <- as.factor(stockm$Direction)

Über die bereits implementierte z - Transformation können die Daten einheitlich skaliert werden. Wichtig dabei ist, dass die Zielvariable, die sich in der 9. Spalte befindet, nicht konvertiert wird.

In [9]:
stockm_new<- as.data.frame(scale(stockm[-9]))

In [10]:
head(stockm_new)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
332,-0.72106947,-1.28384747,3.29679301,-0.26672101,-1.6997547,-0.8989669,-0.9027296,-1.48032029
465,-0.72106947,0.53785640,2.16414990,-0.04366020,0.6764856,-1.8113452,-0.2170208,-0.91886569
715,-0.01135542,-0.01305502,1.00246466,-0.04717297,-0.5098783,-0.4031280,-0.2644738,-0.67333930
1133,1.40807266,0.22719868,0.07399653,0.19784264,0.5473993,1.0129321,1.5828882,0.09932079
252,-0.72106947,-0.55604598,-0.83687030,0.10563246,-0.4229427,-0.3177287,-0.5325412,0.59829377
1119,1.40807266,-0.95646881,0.01591227,-0.18153637,-0.0637844,0.4264653,1.5307455,-0.67421933


Die Transformation scheint geklappt zu haben.  
Nun werden die Daten in Test- und Trainingsdatensätze aufgeteilt.

In [36]:
stockm_train = stockm_new[1:1000, 2:8 ]
stockm_test = stockm_new[1001:1250, 2:8]

Die Klassen müssen in Vektoren abgespeichert werden.

In [37]:
stockm_label_train = stockm[1:1000, 9]
stockm_label_test = stockm[1001:1250, 9]

Nun wird ein Vektor mit den vorhergesagten Klassen für die Testdaten erstellt, indem die kNN - Funktion angewandt wird.

In [42]:
#k=35 Quadratwurzel von 1250
stockm_test_pred <- knn(train=stockm_train, test = stockm_test, cl = stockm_label_train, k=35)

Schließlich überpürfen wir, wie gut die Übereinstimmung mit den vorhergesagten Daten ist. Dazu brauchen wir die library `gmodels`

In [43]:
library(gmodels)
CrossTable(x= stockm_label_test, y=stockm_test_pred, prob.chisq = FLASE) 
#mit prob.chisq = FALSE werden die CHI^2 - Werte nicht angezeigt


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  250 

 
                  | stockm_test_pred 
stockm_label_test |      Down |        Up | Row Total | 
------------------|-----------|-----------|-----------|
             Down |        97 |        21 |       118 | 
                  |    51.042 |    34.599 |           | 
                  |     0.822 |     0.178 |     0.472 | 
                  |     0.960 |     0.141 |           | 
                  |     0.388 |     0.084 |           | 
------------------|-----------|-----------|-----------|
               Up |         4 |       128 |       132 | 
                  |    45.628 |    30.929 |           | 
                  |     0.030 |     0.970 |     0.528 | 
                  |     0.040 |     0.859 |           | 
                  |    

Es wurden knapp 39% korrekt vorhergesagt bei den fallenden Kursen und gut 51% bei den steigenden Kursen. Also liegt diese Modell in unserem Fall bei ca. 90% richtig. 10% wurden falsch klassifiziert.  

#### Die Performance verbessern

Es kann getestet werden, ob mit der Normalisierung der Werte die Vorhersage besser wird. Außerdem kann man unterschiedliche Werte für k testen.

In [46]:
normalize <- function(x){
    return ((x-min(x)) / (max(x) -min(x)))
}

In [47]:
stockm_new <- as.data.frame(lapply(stockm[-9], normalize))

In [48]:
head(stockm_new)

Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
0.25,0.3253871,0.8138902,0.4335992,0.2804317,0.3656499,0.2849843,0.3043641
0.25,0.5196621,0.6931018,0.4574378,0.5343970,0.2673862,0.3733479,0.3642421
0.50,0.4609104,0.5692163,0.4570624,0.4076021,0.4190521,0.3672329,0.3904270
1.00,0.4865321,0.4702018,0.4832473,0.5206007,0.5715626,0.6052925,0.4728297
0.25,0.4030033,0.3730643,0.4733928,0.4168935,0.4282496,0.3326885,0.5260441
1.00,0.3603003,0.4640075,0.4427030,0.4552792,0.5083998,0.5985732,0.3903332


Schließlich teilen wir wieder in Test- und Trainingsdaten auf und speichern die Klassen in Vektoren

In [49]:
stockm_train = stockm_new[1:1000, 2:8 ]
stockm_test = stockm_new[1001:1250, 2:8]

In [50]:
stockm_label_train = stockm[1:1000, 9]
stockm_label_test = stockm[1001:1250, 9]

Dann wird der kNN Algorithmus wieder angewandt.

In [57]:
stockm_test_pred <- knn(train=stockm_train, test = stockm_test, cl = stockm_label_train, k=35)

Und wieder in einer Kreuztabelle die Ergebnisse verglichen.

In [58]:
CrossTable(x= stockm_label_test, y=stockm_test_pred, prob.chisq = FLASE)


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  250 

 
                  | stockm_test_pred 
stockm_label_test |      Down |        Up | Row Total | 
------------------|-----------|-----------|-----------|
             Down |       101 |        17 |       118 | 
                  |    53.391 |    38.663 |           | 
                  |     0.856 |     0.144 |     0.472 | 
                  |     0.962 |     0.117 |           | 
                  |     0.404 |     0.068 |           | 
------------------|-----------|-----------|-----------|
               Up |         4 |       128 |       132 | 
                  |    47.729 |    34.562 |           | 
                  |     0.030 |     0.970 |     0.528 | 
                  |     0.038 |     0.883 |           | 
                  |    

Die Performance konnte um ca 1% verbessert werden, denn nun sind die richtig vorhergesagten fallende Kurse bei 40.4% (vorher bei 38.8 %).

Unterschiedliche k können getestet werden, indem das k bei der `knn()` - Funktion einfach verändert wird und der Code nochmals ausgeführt wird.

In [60]:
stockm_test_pred <- knn(train=stockm_train, test = stockm_test, cl = stockm_label_train, k=1)
CrossTable(x= stockm_label_test, y=stockm_test_pred, prob.chisq = FLASE)


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  250 

 
                  | stockm_test_pred 
stockm_label_test |      Down |        Up | Row Total | 
------------------|-----------|-----------|-----------|
             Down |        98 |        20 |       118 | 
                  |    33.134 |    29.148 |           | 
                  |     0.831 |     0.169 |     0.472 | 
                  |     0.838 |     0.150 |           | 
                  |     0.392 |     0.080 |           | 
------------------|-----------|-----------|-----------|
               Up |        19 |       113 |       132 | 
                  |    29.620 |    26.056 |           | 
                  |     0.144 |     0.856 |     0.528 | 
                  |     0.162 |     0.850 |           | 
                  |    

mit k=1 wird die Performance nicht besser.